#load database (or preporcessing database):
problems: 
*finding database*
*nltk takes matrix as a input* or any machine learning model or in 2 3 array 


#**Important Keynotes:**
>* Activation function (used to make my model's answer either 0 or 1 in case when my model return probability) for that we use non-linear functions such as **relu** or **tanh** etc.
>* dropout layer (**dropout is a one of the regularization technique that prevent model from Overfitting** (performing good on training whereas poorer on new data))
>* some good way to use for loop while building model:
 ```
 for units in [128, 64, 32]:
  dense_tensor = tf.keras.layers.Dense(units, activation='relu')(
    dense_tensor)
 prediction = tf.keras.layers.Dense(1)(dense_tensor)
 ```
>* Converting categorical features to numeric using: 
```
keywords_embedded = tf.feature_column.embedding_column(
     tf.feature_column.categorical_column_with_hash_bucket("keywords", 10000)
```
>* **tf.keras.layers.DenseFetures** layer is used to convert my features into a Dense Tensor or an array
 that can later be fed into model as an input. 
>* It is my choice whether i want to use **tf.keras.layers.DenseFeatures** layer or not because i can manually separate my features and lables which gives me more freedom to tinker them. But if i am manually specifying my features while i am training my mobel i need to specify the shape of my input and output to my neural network's **_input layer_** (or first layer)
 and **_output layer_** simultaneously such as:
```
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200# the deep learning model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)
```

# Source Code

In [ ]:
data = {"intents": [
             {"tag": "greeting",
              "patterns": ["Hello", "How are you?", "Hi there", "Hi", "Whats up"],
              "responses": ["Howdy Partner!", "Hello", "How are you doing?", "Greetings!", "How do you do?"],
             },
             {"tag": "age",
              "patterns": ["how old are you?", "when is your birthday?", "when was you born?"],
              "responses": ["I am 24 years old", "I was born in 1996", "My birthday is July 3rd and I was born in 1996", "03/07/1996"]
             },
             {"tag": "date",
              "patterns": ["what are you doing this weekend?",
"do you want to hang out some time?", "what are your plans for this week"],
              "responses": ["I am available all week", "I don't have any plans", "I am not busy"]
             },
             {"tag": "name",
              "patterns": ["what's your name?", "what are you called?", "who are you?"],
              "responses": ["My name is Kippi", "I'm Kippi", "Kippi"]
             },
             {"tag": "goodbye",
              "patterns": [ "bye", "g2g", "see ya", "adios", "cya"],
              "responses": ["It was nice speaking to you", "See you later", "Speak soon!"]
             }
]}

In [ ]:
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
from matplotlib import pyplot as plt
import pandas as pd

nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()# Each list to create
words = []
classes = []
doc_X = []
doc_y = []# Loop through all the intents
# tokenize each pattern and append tokens to words, the patterns and
# the associated tag to their associated list
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # add the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])
# lemmatize all the words in the vocab and convert them to lowercase
# remove any punctuations
words = [lemmatizer.lemmatize(word.lower()) 
        for word in words 
         if word not in string.punctuation]# sorting the vocab and classes in alphabetical order and
                                           # taking the set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

In [ ]:
print(classes)

['age', 'date', 'goodbye', 'greeting', 'name']


In [ ]:
# list for training data
training = []
out_empty = [0] * len(classes)# creating the bag of words model
for idx, doc in enumerate(doc_X):
    print(idx,doc)
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)# mark the index of class that the current pattern is associated to
    output_row = list(out_empty)
    
    output_row[classes.index(doc_y[idx])] = 1    # add the one hot encoded BoW and associated classes to training 
                                              # it will going to give me [0 0 0 0 0] list value which my model can use for creating pattern
    
    training.append([bow, output_row])
    # shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

0 Hello
1 How are you?
2 Hi there
3 Hi
4 Whats up
5 how old are you?
6 when is your birthday?
7 when was you born?
8 what are you doing this weekend?
9 do you want to hang out some time?
10 what are your plans for this week
11 what's your name?
12 what are you called?
13 who are you?
14 bye
15 g2g
16 see ya
17 adios
18 cya


In [ ]:
def plot_the_loss_curve(epochs, mse):
  """Plot a curve of loss vs. epoch."""

  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Mean Squared Error")

  plt.plot(epochs, mse, label="Loss")
  plt.legend()
  plt.ylim([mse.min()*0.95, mse.max() * 1.03])
  plt.show()  

print("Defined the plot_the_loss_curve function.")

Defined the plot_the_loss_curve function.


In [ ]:
from nltk.translate import metrics
# defining some parameters
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
def create_model(my_learning_rate):
  model = Sequential()
  model.add(Dense(128, input_shape=input_shape, activation="relu"))
  model.add(Dropout(0.5))
  model.add(Dense(64, activation="relu"))
  model.add(Dropout(0.3))
  model.add(Dense(output_shape, activation = "softmax"))
  adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
  model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
  return model

In [ ]:
def train_model(model, batch_size, epochs):
  featurs = train_X
  labels = train_y
  history = model.fit(x=featurs, y=labels, batch_size=batch_size,
            epochs=epochs, shuffle=True)
  epochs = history.epoch

  # To track the progression of training, gather a snapshot
  # of the model's mean squared error at each epoch. 
  hist = pd.DataFrame(history.history)
  accuracy = hist["Accuracy"]

  return epochs, accuracy

In [ ]:
# defining some parameters
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200# the deep learning model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               5120      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 5)                 325       
                                                                 
Total params: 13,701
Trainable params: 13,701
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
1/1 [==============================

In [ ]:
def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.2
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]

  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list

def get_response(intents_list, intents_json): 
  tag = intents_list[0]
  list_of_intents = intents_json["intents"]
  for i in list_of_intents: 
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result

In [ ]:
# running the chatbot
while True:
    message = input("")
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)
    if message == "bye":
      break
  